In [ ]:
import cv2
import math
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from torchvision import transforms
import random
import torch

video_legal_dir = r"D:\Video-Dataset\2022-NSR-v4-after-3m\agree"
video_illegal_dir = r"D:\Video-Dataset\2022-NSR-v4-after-3m\non-agree"

def transform_diagonal(image, mean = 0, std = 0, is_normalize = False):
    if is_normalize:
        transform = transforms.Compose([
            transforms.ToTensor(), 
            transforms.Normalize(mean, std)
        ])

        return transform(image)
    else:
        transform = transforms.Compose([
            transforms.ToTensor(), 
        ])

        return transform(image)

def extract_diagnoal_matrix(dir_path : str):
    video_paths = glob.glob(dir_path + "\*.mp4")
    video_paths.sort()
    video_diagonals = []
    video_names = []

    pre_ht = []

    for video_path in tqdm(video_paths, total=len(video_paths), desc="Extract Features"):
        vidcap = cv2.VideoCapture(video_path)
        vid_total_frames_num = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
        vid_frame_per_s = int(vidcap.get(cv2.CAP_PROP_FPS))
        video_name = video_path.split("\\")[-1]

        sections, retstep = np.linspace(1, vid_total_frames_num, 256, retstep=True)
        sections = list(map(math.floor, sections))
        frame_diagonals = []
        frames = []

        while(vidcap.isOpened()):
            ret, frame = vidcap.read()
            
            if not ret:
                break

            if int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)) in sections:
                frame = cv2.resize(frame, (256, 256))

                # if int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)) == 1:
                #     pre_frame = frame
                #     continue
                
                # frame_df = cv2.absdiff(pre_frame, frame)

                frame_r, frame_g, frame_b = frame[:,:,0], frame[:,:,1], frame[:,:,2]
                frame_r, frame_g, frame_b = np.diag(frame_r), np.diag(frame_g), np.diag(frame_b)

                frame_diagonal = np.stack([frame_r, frame_g, frame_b], -1)
                frame_diagonal = np.expand_dims(frame_diagonal, 1)
                frame_diagonals.append(frame_diagonal)
                # frame_diagonals.append(frame_df)
                # pre_frame = frame

        vidcap.release()
        video_diagonal = np.concatenate(frame_diagonals, axis=1)

        # video_diagonal = transform_diagonal(video_diagonal.copy())
        # video_diagonal_tr = transform_diagonal(video_diagonal.copy())
        # mean, std = video_diagonal_tr.mean([1,2]), video_diagonal_tr.std([1,2])
        # video_diagonal = transform_diagonal(video_diagonal, mean, std, True)
        # video_diagonal = transform_diagonal(video_diagonal, (0.485, 0.456, 0.406), (0.229, 0.224, 0.225), True)
                            
        video_diagonals.append(video_diagonal)
        video_names.append(video_name)
    
    return video_names, video_diagonals

legal_diagonals = extract_diagnoal_matrix(video_legal_dir)
illegal_diagonals = extract_diagnoal_matrix(video_illegal_dir)

In [ ]:
diagonals_legal = []
diagonals_illegal = []
legal_names = []
illegal_names = []

for name, diagonal_legal in zip(legal_diagonals[0], legal_diagonals[1]):
    legal_names.append(name)
    diagonals_legal.append(diagonal_legal)

for name, diagonal_illegal in zip(illegal_diagonals[0], illegal_diagonals[1]):
    illegal_names.append(name)
    diagonals_illegal.append(diagonal_illegal)

In [ ]:
for n, i in zip(illegal_names, diagonals_illegal):
    # img_tr = transform_tensor(i)
    # mean, std = i.mean([1,2]), i.std([1,2])
    # img_nor = i
    # i = np.array(i.permute(1, 2, 0))

    # print(f"mean: {mean}, std: {std}")
    # i = i.astype(int)
    plt.imshow(i)
    plt.show()

    plt.hist(np.array(i).ravel(), bins=50, density=True)
    plt.xlabel("pixel values")
    plt.ylabel("relative frequency")
    plt.title("distribution of pixels")
    plt.show()

In [ ]:
from PIL import Image

save_dir = r"D:\Video-Dataset\v4-image\non-agree"

def image_save(images):
    for name, image in tqdm(zip(images[0], images[1]), total=len(images[0]), desc="Image save"):
        name = name.split(".")[0] + ".png"
        image = Image.fromarray(image)
        image.save(f"{save_dir}\{name}")

# image_save(legal_diagonals)
image_save(illegal_diagonals)

In [ ]:
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
 
# define custom transform function
transform = transforms.Compose([
    transforms.ToTensor()
])
 
# transform the pIL image to tensor
# image

img_tr = []
for i in x:
    print(i.shape)
    img_tr.append(transform(i))
    
# Convert tensor image to numpy array

img_np = np.array(img_tr[1])

# plot the pixel values
plt.hist(img_np.ravel(), bins=50, density=True)
plt.xlabel("pixel values")
plt.ylabel("relative frequency")
plt.title("distribution of pixels")

In [ ]:
import torch
for img in (x):
    # img_tr = transform(img)

    img_tr = torch.Tensor(img/255.0)
    
    # calculate mean and std
    mean, std = img_tr.mean([1,2]), img_tr.std([1,2])
    
    # print mean and std
    print("mean and std before normalize:")
    print("Mean of the image:", mean)
    print("Std of the image:", std)
    print()
    break

In [ ]:
transform_norm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    # transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
 
# get normalized image
img_normalized = transform_norm(x[0])
 
# convert normalized image to numpy
# array
img_np = np.array(img_normalized)
 
# plot the pixel values
plt.hist(img_np.ravel(), bins=50, density=True)
plt.xlabel("pixel values")
plt.ylabel("relative frequency")
plt.title("distribution of pixels")

In [ ]:
img_nor = transform_norm(x[0])
 
# cailculate mean and std
mean, std = img_nor.mean([1,2]), img_nor.std([1,2])
 
# print mean and std
print("Mean and Std of normalized image:")
print("Mean of the image:", mean)
print("Std of the image:", std)